In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


créé un dossier music-generation-colab dans ton gg drive et mets y le dossier data

In [2]:
import os
os.chdir("drive/My Drive/music-generation-colab")

In [3]:
import pandas as pd
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


def prepare_sequences(notes, n_vocab, sequence_length = 100):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = sequence_length

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number)
                       for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(
        network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)


def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model


def train(model, network_input, network_output, epochs=2, batch_size=128):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=epochs,
              batch_size=batch_size, callbacks=callbacks_list)




In [74]:
pitchnames = sorted(set(item for item in notes))
dict((note, number) for number, note in enumerate(pitchnames))

{'A0': 0,
 'A0.A1': 1,
 'A0.B-0': 2,
 'A1': 3,
 'A1.A0': 4,
 'A1.A2': 5,
 'A1.A2.A4': 6,
 'A1.A2.B2': 7,
 'A1.A2.D4': 8,
 'A1.A2.G2': 9,
 'A1.B1': 10,
 'A1.C#4': 11,
 'A1.F#1.B1': 12,
 'A1.G2.B5': 13,
 'A2': 14,
 'A2.A1': 15,
 'A2.A1.B1': 16,
 'A2.A1.D2': 17,
 'A2.A1.D4': 18,
 'A2.A1.F#2': 19,
 'A2.A1.G3.G2.G#1.B3.B-1.G#3': 20,
 'A2.A3': 21,
 'A2.A3.A4.A1': 22,
 'A2.A4.A1': 23,
 'A2.A4.A3': 24,
 'A2.B-1.A1': 25,
 'A2.B-2.E3': 26,
 'A2.B1': 27,
 'A2.B1.A1.B-1': 28,
 'A2.B1.A3': 29,
 'A2.B1.B2.C2': 30,
 'A2.B1.C2': 31,
 'A2.B2': 32,
 'A2.C#2': 33,
 'A2.C#3': 34,
 'A2.C#4.A1': 35,
 'A2.C2.B2': 36,
 'A2.C3': 37,
 'A2.C3.E-3': 38,
 'A2.C5.A1.E-4.F#4.C4': 39,
 'A2.D2': 40,
 'A2.D3': 41,
 'A2.D3.D2': 42,
 'A2.D5': 43,
 'A2.E-2.D3.D4': 44,
 'A2.E-3.F#3': 45,
 'A2.E2': 46,
 'A2.E3': 47,
 'A2.E3.D3': 48,
 'A2.E4': 49,
 'A2.E4.B-4.A1': 50,
 'A2.F#3.C#3.A5.A6.G3.B-2': 51,
 'A2.F3': 52,
 'A2.G2': 53,
 'A2.G2.D3': 54,
 'A2.G2.G1.A1': 55,
 'A2.G3.A3': 56,
 'A3': 57,
 'A3.A1': 58,
 'A3.A2': 59,
 'A3.A

In [4]:
notes = []
log = []

annees = [2002]
notes_source = []
for x in annees:
    notesAnnees = pd.read_pickle(r'data/results/notes_' + str(x))
    notes_source += notesAnnees
notes_source = notes_source[0:1000]


# test avec entrée = alternance de 2 notes


In [59]:
notes = notes_source
n_vocab = len(set(notes))
print(notes[0:10])

['B-3.F3.D3', 'B-2.B-1', 'C#3', 'C#2', 'B-3.F3.D3', 'B-2.B-1', 'D3.F3.B-2', 'D3.B-2.F3', 'F3.D3', 'B-2']


In [62]:
print('creating inputs')
network_input, network_output = prepare_sequences(notes, n_vocab, sequence_length=10)


print('creating model for training')
model = create_network(network_input, n_vocab)

print('training model')
train(model, network_input, network_output,epochs=100, batch_size=128)

print('model trained')

creating inputs
creating model for training
training model
Epoch 1/100
79/79 [==============================] - 7s 46ms/step - loss: 6.5056
Epoch 2/100
79/79 [==============================] - 4s 46ms/step - loss: 6.0118
Epoch 3/100
79/79 [==============================] - 4s 47ms/step - loss: 6.0225
Epoch 4/100
79/79 [==============================] - 4s 52ms/step - loss: 5.9733
Epoch 5/100
79/79 [==============================] - 4s 46ms/step - loss: 5.9507
Epoch 6/100
79/79 [==============================] - 3s 44ms/step - loss: 5.9339
Epoch 7/100
79/79 [==============================] - 4s 45ms/step - loss: 5.9171
Epoch 8/100
79/79 [==============================] - 4s 50ms/step - loss: 5.9318
Epoch 9/100
79/79 [==============================] - 4s 48ms/step - loss: 5.8871
Epoch 10/100
79/79 [==============================] - 4s 45ms/step - loss: 5.8913
Epoch 11/100
79/79 [==============================] - 4s 48ms/step - loss: 5.8976
Epoch 12/100
79/79 [============================

In [63]:
print('generating notes')
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(set(notes)))
pattern = network_input[start]
prediction_output = []
# generate 10 notes
for note_index in range(20):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern,index/ float(n_vocab))
    pattern = pattern[1:len(pattern)]
    
print(prediction_output)

generating notes
['F4.C4.D4', 'E-5.G4', 'E-1', 'E-1', 'A1.A2', 'B-3.D4.G#3.C#4', 'C#3.G#2.G#3.C#4.F3.C#2', 'E-3.G3.A3.A4.G4.D5.C3.B-2', 'C6.C5.G#2.B-1.E5.G#5.G#1.A1', 'E-3.G3.A3.A4.G4.D5.C3.B-2', 'E-3.G3.A3.A4.G4.D5.C3.B-2', 'G#3.G#4.D4.D3.B3', 'C#3.G#2.G#3.C#4.F3.C#2', 'A2.E-2.D3.D4', 'E-3.G3.A3.A4.G4.D5.C3.B-2', 'E-5.G4', 'E-3.G3.A3.A4.G4.D5.C3.B-2', 'E-3.G3.A3.A4.G4.D5.C3.B-2', 'E-5.G4', 'E-3.G3.A3.A4.G4.D5.C3.B-2']


In [66]:
pattern = network_input[start]


In [70]:
prediction.shape

(1, 2324)

In [67]:
pattern

array([[0.02452668],
       [0.89845095],
       [0.16049914],
       [0.02538726],
       [0.15017212],
       [0.02538726],
       [0.82831325],
       [0.89845095],
       [0.0292599 ],
       [0.00903614]])

In [68]:
numpy.reshape(pattern, (1, len(pattern), 1))

array([[[0.02452668],
        [0.89845095],
        [0.16049914],
        [0.02538726],
        [0.15017212],
        [0.02538726],
        [0.82831325],
        [0.89845095],
        [0.0292599 ],
        [0.00903614]]])

In [65]:
!pip install midiutil

In [48]:
!pip install music21

In [64]:
from midiutil import MIDIFile
from music21 import *

degrees  = pitch_to_midi_number(prediction_output)  # MIDI note number
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = 60   # In BPM
volume   = 100  # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
                      # automatically)
MyMIDI.addTempo(track, time, tempo)

for i, pitch in enumerate(degrees):
    if type(pitch) is list:
        for p in pitch:
            MyMIDI.addNote(track, channel, p, time + i, duration, volume)
    else:
        MyMIDI.addNote(track, channel, pitch, time + i, duration, volume)

with open("major-scale.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

In [55]:
def pitch_to_midi_number(pattern):
    midi_notes = []
    for p in pattern:
        if ('.' in p):
            notes_in_chord = p.split('.')
            notes = []
            for current_note in notes_in_chord:
                notes.append(pitch.Pitch(current_note).midi)
            midi_notes.append(notes)
        else:
            midi_notes.append(pitch.Pitch(p).midi)
    return midi_notes